In [37]:
import pandas as pd

bank = pd.read_excel("D:/CDAC/Project/AI-Driven-Personal-Financial-Risk-Spending-Behavior-Analyzer/Dataset/bank.xlsx")
personal = pd.read_csv("D:/CDAC/Project/AI-Driven-Personal-Financial-Risk-Spending-Behavior-Analyzer/Dataset/personal_transactions.csv")


In [38]:
bank.columns = bank.columns.str.strip()

bank.rename(columns={
    'DATE':'Date',
    'BALANCE AMT':'Balance'
}, inplace=True)

bank = bank.loc[:,~bank.columns.duplicated()]

bank['Type'] = bank.apply(lambda x: 'Debit' if x['WITHDRAWAL AMT']>0 else 'Credit', axis=1)

bank['Amount'] = bank['WITHDRAWAL AMT']
bank['Amount'] = bank['Amount'].replace(0, pd.NA)
bank['Amount'] = bank['Amount'].fillna(bank['DEPOSIT AMT'])

bank['Category'] = 'bank'
bank['Balance'] = pd.to_numeric(bank['Balance'], errors='coerce')

bank = bank[['Date','Type','Amount','Category','Balance']]
bank['Date'] = pd.to_datetime(bank['Date'], errors='coerce')


In [39]:
personal.rename(columns={
    'Transaction Type':'Type',
    'Account Name':'Account'
}, inplace=True)

personal['Type'] = personal['Type'].str.lower().map({
    'debit':'Debit','expense':'Debit',
    'credit':'Credit','income':'Credit'
})

personal['Date'] = pd.to_datetime(personal['Date'], errors='coerce')
personal['Category'] = personal['Category'].str.lower().str.strip()
personal['Balance'] = None

personal = personal[['Date','Type','Amount','Category','Balance']]


In [40]:
df = pd.concat([bank, personal], ignore_index=True)
df = df.sort_values(by='Date').reset_index(drop=True)
df['Month'] = df['Date'].dt.to_period('M')


C:\Users\Admin\AppData\Local\Temp\ipykernel_16056\3019026753.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([bank, personal], ignore_index=True)


In [41]:
income = df[df['Type']=='Credit'].groupby('Month')['Amount'].sum()
expense = df[df['Type']=='Debit'].groupby('Month')['Amount'].sum()
transactions = df.groupby('Month')['Amount'].count()
avg_exp = df[df['Type']=='Debit'].groupby('Month')['Amount'].mean()
max_exp = df[df['Type']=='Debit'].groupby('Month')['Amount'].max()
low_balance = df.groupby('Month')['Balance'].apply(lambda x: (x<2000).sum())


In [42]:
monthly = pd.DataFrame({
    'Total_Income': income,
    'Total_Expense': expense,
    'Num_Transactions': transactions,
    'Avg_Expense': avg_exp,
    'Max_Expense': max_exp,
    'Low_Balance_Freq': low_balance
}).fillna(0).reset_index()

monthly['Expense_Income_Ratio'] = monthly['Total_Expense'] / monthly['Total_Income']
monthly.replace([float('inf')], 0, inplace=True)


In [43]:
cat = df[df['Type']=='Debit'].groupby(['Month','Category'])['Amount'].sum().reset_index()
top_cat = cat.loc[cat.groupby('Month')['Amount'].idxmax()]
top_cat.rename(columns={'Amount':'Top_Category_Spend'}, inplace=True)

monthly = monthly.merge(top_cat[['Month','Top_Category_Spend']], on='Month', how='left').fillna(0)


In [44]:
def risk(row):
    if row['Expense_Income_Ratio'] > 0.9 or row['Low_Balance_Freq'] > 5:
        return "High"
    elif row['Expense_Income_Ratio'] > 0.7:
        return "Medium"
    else:
        return "Low"

monthly['Risk_Label'] = monthly.apply(risk, axis=1)


In [45]:
monthly.to_csv("D:/CDAC/Project/AI-Driven-Personal-Financial-Risk-Spending-Behavior-Analyzer/Dataset/final_financial_risk_dataset.csv", index=False)